# 기타 분석

>* Tx 분류 - Large, Big transactions
>* 플랫폼 공격 여부 - (0 value transactions)
>* 휴면계정 - active한 계정이 몇 개인지, 0 balance accounts 수
>* 부의 분배가 평등한지

In [1]:
category = 'LINK'

In [2]:
file_1 = "_rawdata.txt"
file_2 = "_price.txt"
file_3 = "_transfer.txt"
file_4 = "_account_distribution_count_1.txt"
file_5 = "_account_distribution_sum_1.txt"
file_6 = "_account_stats.txt"

In [3]:
file_path = category + file_1
file_path_2 = category + file_6

In [4]:
import pandas as pd

In [5]:
data = pd.read_csv(file_path, sep="\t", header=None)
data.columns = ["TXID", "Date", "Unix Timestamp", "From Address", "To Address", "Amount"]
data['Unix Timestamp'] = pd.to_datetime(data['Unix Timestamp'], unit='s')

In [6]:
data2 = pd.read_csv(file_path_2, sep="\t", header=None)
data2.columns = ["Account", "Balance", "In_Degree", "Out_Degree", "Recent Seen (UnixTimeStamp)"]
data2['Recent Seen (UnixTimeStamp)'] = pd.to_datetime(data2['Recent Seen (UnixTimeStamp)'], unit='s')

In [7]:
data2 = pd.DataFrame(data2)

In [8]:
FromAdd = data.drop('To Address', axis=1)
FromAdd['Amount'] *= -1
FromAdd.rename(columns={'From Address': 'Address'}, inplace=True)

In [9]:
ToAdd = data.drop('From Address', axis=1)
ToAdd.rename(columns={'To Address': 'Address'}, inplace=True)

In [10]:
output = FromAdd.append(ToAdd)

In [11]:
result = output.groupby(['Address']).size().reset_index(name='counts')

In [12]:
result = output.merge(result)

In [13]:
result = result.sort_values(by=['counts'], ascending=False)

### Transactions categorization

large_transactions = transaction이 q3보다 많이 일어난 경우

big_transactions = transaction의 금액이 q3보다 큰 경우

In [14]:
large_transactions = result[result['counts'] > result.counts.quantile(0.85)]

In [15]:
big_transactions = result[result['Amount'] > result.Amount.quantile(0.85)]

In [16]:
print("large transactions : ", "%.2f" % (len(large_transactions) / len(data2.Account) * 100) , "%")
print("big_transactions : ", "%.2f" % (len(big_transactions) / len(data2.Account) * 100), "%")

large transactions :  78.71 %
big_transactions :  83.19 %


### Is there any unsafe transaction?

In [17]:
division = data.groupby(['From Address', 'Date']).size().reset_index(name='Division')
boxplot4 = division.boxplot(column='Division', showfliers=True)

In [18]:
import numpy as np
import pandas as pd
outliers=[]
def detect_outlier(data_):
    
    threshold = 50
    mean_ = np.mean(data_)
    std_ = np.std(data_)
    
    normalized_ = (data_ - mean_) / std_
    outliers = normalized_[normalized_ > threshold]
    
    return outliers

In [19]:
outliers = detect_outlier(division['Division']).index.tolist()

In [20]:
columns = data.columns.tolist()
df_ = pd.DataFrame(columns=columns)

In [21]:
for index in outliers:
    d = data[data['From Address'] == division.iloc[index]['From Address']]
    frames = [df_, d]
    df_ = pd.concat(frames) 
df_[df_['Amount'] == 0]

,TXID,Date,Unix Timestamp,From Address,To Address,Amount
77920,0x0eb9eba0bdb4fe3fcdb92678b9c5ec22b627459ef889...,2017-12-26,2017-12-26 12:21:29,0x8d12a197cb00d4747a1fe03395095ce2a5cc6819,0xa7fd6bf6c5d09ff38fdd1a6643ea1baf3b254650,0.0
82357,0x9f3d218825a404a17179e922e6ae460e97391e1ba608...,2018-01-04,2018-01-03 15:10:13,0x8d12a197cb00d4747a1fe03395095ce2a5cc6819,0x25f59655bf3ae6eb309c20e609d6320b10aede1b,0.0


### Active Accounts

기준 : 최근 활동이 1년 이내인 경우 Active한 계정

In [22]:
import datetime
Inactive = data2[data2["Recent Seen (UnixTimeStamp)"] < (datetime.datetime.now() - datetime.timedelta(days=365))]
Inactive_accounts = len(Inactive)/len(data2)*100
print("Inactive accounts :", "%.2f" % Inactive_accounts, "%")

Inactive accounts : 60.28 %


In [23]:
zero_balance_inactive = len(Inactive[Inactive['Balance'] == 0]) / len(Inactive) * 100
print("Number of 0 balance accounts in inactive accounts :", "%.2f" % zero_balance_inactive, "%")

Number of 0 balance accounts in inactive accounts : 53.74 %


### Concentration of Wealth - Gini Coefficient

In [24]:
def gini(x, w=None):
    # w = weight
    import numpy as np
    x = np.asarray(x)
    if w is not None:
        w = np.asarray(w)
        sorted_indices = np.argsort(x)
        sorted_x = x[sorted_indices]
        sorted_w = w[sorted_indices]
        # Force float dtype to avoid overflows
        cumw = np.cumsum(sorted_w, dtype=float)
        cumxw = np.cumsum(sorted_x * sorted_w, dtype=float)
        return (np.sum(cumxw[1:] * cumw[:-1] - cumxw[:-1] * cumw[1:]) / 
                (cumxw[-1] * cumw[-1]))
    else:
        sorted_x = np.sort(x)
        n = len(x)
        cumx = np.cumsum(sorted_x, dtype=float)
        # The above formula, with all weights equal to 1 simplifies to:
        return (n + 1 - 2 * np.sum(cumx) / cumx[-1]) / n

In [25]:
gini(data2['Balance'])

0.9897280438998994

### Concentration of Wealth - Visualization

In [26]:
top_10 = data2.sort_values(by='Balance', ascending=False).iloc[0:10].Balance.sum()

In [27]:
top_100 = data2.sort_values(by='Balance', ascending=False).iloc[10:100].Balance.sum()

In [28]:
top_1000 = data2.sort_values(by='Balance', ascending=False).iloc[100:1000].Balance.sum()

In [29]:
bottom = data2.sort_values(by='Balance', ascending=False).iloc[1000:].Balance.sum()

In [30]:
from pip._internal import main as pipmain

pipmain(['install', 'plotly'])

0

In [31]:
import plotly.plotly as py
import plotly.graph_objs as go

labels = ['top10', '10-100', '100-1k', 'bottom']
values = [top_10, top_100, top_1000, bottom]
colors = ['gold', 'yellowgreen', 'lightcoral', 'lightskyblue']

trace = go.Pie(labels=labels, values=values,
               hoverinfo='label+value', textinfo='percent',
               opacity = 0.8,
               textfont=dict(size=20),
               marker=dict(colors=colors))


py.iplot([trace], filename=category + " wealth distribution")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



### Phish, Hack

Etherscan Label 이용

In [32]:
import requests
from bs4 import BeautifulSoup

def label_lst(url):
    response = requests.get(url)
    html = response.text
    soup=BeautifulSoup(html,'html.parser')

    columns=soup.select('div.table-responsive > table > thead > tr > th')
    columnlist=[]
    for column in columns:
        columnlist.append(column.text)

    contents=soup.select('div.table-responsive > table > tbody > tr')
    dfcontent=[]
    alldfcontents=[]

    for content in contents:
        tds=content.find_all("td")
        for td in tds:
            dfcontent.append(td.text)
        alldfcontents.append(dfcontent)
        dfcontent=[]
    
    df = pd.DataFrame(columns=columnlist, data=alldfcontents)
    
    return df

In [33]:
url = "https://etherscan.io/accounts/1?ps=100&l=Phish%2fHack"
label = label_lst(url)

for number in range(2,29):
    url = "https://etherscan.io/accounts/" + str(number) + "?ps=100&l=Phish%2fHack"
    label1 = label_lst(url)
    frames = [label, label1]
    label = pd.concat(frames)

In [34]:
label.columns = ['Index', 'Address', 'Label', 'Balance ', 'TxCount']
del label['Index']

In [35]:
phish = data[data['From Address'].isin(label.Address.tolist())]
hack = data[data['To Address'].isin(label.Address.tolist())]
frames = [phish, hack]
phish_hack = pd.concat(frames)

In [36]:
print("Phish and Hack transactions :", "%.2f" % (len(phish_hack) / len(data) * 100), "%")

Phish and Hack transactions : 0.05 %
